In [1]:
import pandas as pd
from datetime import datetime
from common import fetch_data, get_backtest_result, get_hodl_result, max_drawdown, get_cache_path, save_cache_data, load_cache_data

In [2]:
# Inputs
exchanges_markets = [
    {
        "binance": "BTCUSDT",
        "bitmex": "XBTUSD",
        "apollox": "BTCUSDT",
        "drift": "BTC-PERP"
    },
    {
        "binance": "ETHUSDT",
        "bitmex": "ETHUSD",
        "apollox": "ETHUSDT",
        "drift": "ETH-PERP"
    },
    {
        "binance": "SOLUSDT",
        "bitmex": "SOLUSD",
        "apollox": "SOLUSDT",
        "drift": "SOL-PERP"
    },
    {
        "binance": "XRPUSDT",
        "bitmex": "XRPUSD",
        "apollox": "XRPUSDT",
        "drift": "XRP-PERP"
    },
    {
        "binance": "BNBUSDT",
        "bitmex": "BNBUSD",
        "apollox": "BNBUSDT",
        "drift": "BNB-PERP"
    },
]

In [3]:
# Compute funding rate PNL with only one market
accepted_exchanges = ["binance", "apollox", "drift"]
exchanges_markets = [{key: market[key] for key in market if key in accepted_exchanges} for market in exchanges_markets]

benchmark_exchange = "binance"

# Initial variables
min_time = 0
max_time = datetime.now().timestamp()

data_df_map = {}
hodl_df_map = {}
results = {}

leverage = 1

for exchanges_market in exchanges_markets:
    min_time = 0
    max_time = datetime.now().timestamp()

    # Read data from file
    for exchange, market in exchanges_market.items():
        data_df = load_cache_data(exchange, market)
        if (market not in data_df_map):
            data_df_map[market] = {}
        data_df_map[market][exchange] = data_df

    # calculate min max time
    for exchange, market in exchanges_market.items():
        data_df = data_df_map[market][exchange]
        min_time = max(min_time, data_df["timestamp"].min())
        max_time = min(max_time, data_df["timestamp"].max())

    # Fetch all funding rates and check date range
    for exchange, market in exchanges_market.items():
        data_df = data_df_map[market][exchange]
        data_df['datetime'] = data_df['datetime'].apply(lambda x: pd.to_datetime(x))

        data_df = data_df[
            (data_df["timestamp"] >= min_time) & (data_df["timestamp"] <= max_time)
        ]

        data_df = data_df.sort_values(by='datetime', ascending=True)
        data_df = data_df.reset_index(drop=True)

        result_df = get_backtest_result(data_df, leverage)
        if (market not in results):
            results[market] = {}
        results[market][exchange] = result_df

    benchmark_market = exchanges_market[benchmark_exchange]
    hodl_df_map[benchmark_market] = get_hodl_result(results[benchmark_market][benchmark_exchange])

# Results are store in 'results' and 'hodl_df_map' variables

/Users/sainy/Documents/projects/funding/funding-arb-analysis/common.py:89: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.999' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[index, 'clt'] = max(new_clt, 0)
/Users/sainy/Documents/projects/funding/funding-arb-analysis/common.py:91: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '28350.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[index, 'entry'] = price if traded else prev_df['entry']
/Users/sainy/Documents/projects/funding/funding-arb-analysis/common.py:93: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '28322.1495' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[index, 'pos_size'

In [ ]:
# Summary table

accepted_exchanges = ["binance", "bitmex", "apollox", 'drift']
exchanges_markets = [{key: market[key] for key in market if key in accepted_exchanges} for market in exchanges_markets]
benchmark_exchange = "binance"

result = []

for exchanges_market in exchanges_markets:
    market = exchanges_market[benchmark_exchange]
    hodl_df = hodl_df_map[market]

    row = {}
    row['symbol'] = market
    row['exchange'] = f'{benchmark_exchange} hodl'
    row['start_date'] = hodl_df.iloc[0]['datetime']
    row['end_date'] = hodl_df.iloc[-1]['datetime']
    row['num_date'] = (row['end_date'] - row['start_date']).days

    row['pnl'] = hodl_df.iloc[-1]['pnl'] * 100
    row['max_drawdown'] = max_drawdown(hodl_df['pnl']) * 100
    row['corr'] = hodl_df['pnl'].corr(hodl_df['pnl'])

    row['annualized_pnl'] = row['pnl'] / row['num_date'] * 365
    result.append(row)

    for exchange, market in exchanges_market.items():

        data_df = results[market][exchange]
        
        row = {}
        row['symbol'] = market
        row['exchange'] = exchange
        row['start_date'] = data_df.iloc[0]['datetime']
        row['end_date'] = data_df.iloc[-1]['datetime']
        row['num_date'] = (row['end_date'] - row['start_date']).days

        row['pnl'] = data_df.iloc[-1]['final_pnl'] * 100
        row['max_drawdown'] = max_drawdown(data_df['final_pnl']) * 100
        row['corr'] = data_df['final_pnl'].corr(hodl_df['pnl'])

        row['annualized_pnl'] = row['pnl'] / row['num_date'] * 365

        result.append(row)

result_df = pd.DataFrame(result)
result_df.to_csv("./output/basic_funding.csv")

print(result_df)